# Comparison of Regressors for Abalone Age Prediction

## The dataset

### What is the purpose of the dataset? 
### What are the features
### What is the target?


## Analyzing the dataset

## Pre-proccessing 

## Splitting

### Why this split percentage
### Why this sampling technique

## Tuning the hyperparameters

### Grid search vs Random search

## The Results


## Comparison of the classifiers



## Conclusion